In [1]:
import pandas as pd
from itertools import combinations
from math import pi, sin, cos
import fileinput

In [2]:
core_df = pd.read_csv('bm_core.csv')
core_df.head()

,Авторы,Заглавие,Издание,Тип публикации
0,Корчак Владимир Юрьевич ...,Автоматизация планово-производственной деятель...,Компетентность\n \n 2016\n .-...,Статья в журнале
1,Верба Владимир Степанович ...,Авиационные комплексы радиолокационного дозора...,Информационно-измерительные и управляющие сист...,Статья в журнале
2,Зайцев Александр Вячеславович ...,Автоматизация поддержки решений при подготовке...,Известия высших учебных заведений. Машинострое...,Статья в журнале
3,Кравченко Виктор Филиппович ...,Автограф Виктора Петровича Шестопалова,Физические основы приборостроения\n \n ...,Статья в журнале
4,Майорова Вера Ивановна ...,Автоматизация приема и обработки резервной тел...,"Вестник МГТУ им.Н.Э.Баумана. Серия ""Машиностро...",Статья в журнале


In [3]:
has_scopus_name = lambda row: False if row['Издание'].find('Scopus') == -1 else True
core_df['Scopus'] = core_df.apply(has_scopus_name, axis=1)
core_df = core_df[core_df['Scopus']]

In [4]:
core_df.head()

,Авторы,Заглавие,Издание,Тип публикации,Scopus
66,Филиппов Сергей Сергеевич ...,АВС-схемы для численного решения жестких систе...,"Вестник МГТУ им.Н.Э.Баумана. Серия ""Естественн...",Статья в журнале,True
28,Seregina E.V. \n(Ка...,Analysis of the Three-Dimensional Model of Dif...,"Journal of Surface Investigation: X-ray, Synch...",Статья в журнале,True
30,Kristya V.I. \n(Кал...,Influence of a cathode surface oxide film on t...,"Journal of Surface Investigation: X-ray, Synch...",Статья в журнале,True
31,Spiridonov I.N. \n(...,Detection of Dermatoglyphic Predictors of Sex ...,Biomedical Engineering\n \n 2014\n ...,Статья в журнале,True
35,Белянчиков М.А. \n ...,Инфракрасная спектроскопия и комбинационное ра...,"Вестник МГТУ им.Н.Э.Баумана. Серия ""Естественн...",Статья в журнале,True


In [5]:
stucked_names = core_df['Авторы']
stucked_names.head()

66    Филиппов Сергей Сергеевич                     ...
28    Seregina E.V.                            \n(Ка...
30    Kristya V.I.                            \n(Кал...
31    Spiridonov I.N.                            \n(...
35    Белянчиков М.А.                            \n ...
Name: Авторы, dtype: object

In [6]:
def split_names(stucked_names_entry):
    splitted = stucked_names_entry.split('\n')
    str_list = []
    for substr in splitted:
        temp = ' '.join(substr.split())
        str_list.append(temp)
    str_list = list(filter(None, str_list))
    return str_list

In [7]:
def pairwise(t):
    it = iter(t)
    return zip(it,it)

In [8]:
def detect_faculty(str_with_faculty):
    selector = {
        str_with_faculty.find('МТ'): 'МТ',
        str_with_faculty.find('ИУ'): 'ИУ',
        str_with_faculty.find('РЛ'): 'РЛ',
        str_with_faculty.find('ФН'): 'ФН',
        str_with_faculty.find('СМ'): 'СМ',
        str_with_faculty.find('Э'): 'Э',
        str_with_faculty.find('РК'): 'РК',
        str_with_faculty.find('БМТ'): 'БМТ',
        str_with_faculty.find('Л'): 'Л',
        str_with_faculty.find('ИБМ'): 'ИБМ',
        str_with_faculty.find('СГН'): 'СГН',
        str_with_faculty.find('РКТ'): 'РКТ',
        str_with_faculty.find('АК'): 'АК',
        str_with_faculty.find('ПС'): 'ПС',
        str_with_faculty.find('РТ'): 'РТ',
        str_with_faculty.find('ОЭП'): 'ОЭП',
        str_with_faculty.find('ЮР'): 'ЮР',
        str_with_faculty.find('ГУИМЦ'): 'ГУИМЦ',
        str_with_faculty.find('ФМОП'): 'ФМОП',
        str_with_faculty.find('ФВО'): 'ФВО',
    }
    try:
        sel = selector[True]
        assert(type(sel) == str), 'Multiple faculties detected: {}'.format(type(sel))
        return sel
    except KeyError as e:
        return 'Other'

In [9]:
def process_group_of_names_df(group_of_names, personal_publications_df, pair_publications_df):
    splitted = split_names(group_of_names)
    name_position_pairs = pairwise(splitted)
    bmstu_members = [entry for entry in name_position_pairs if (entry[1].find('Не МГТУ') == -1)]
    
    
    
def process_group_of_names(group_of_names):
    splitted = split_names(group_of_names)
    name_position_pairs = pairwise(splitted)
    bmstu_members = [entry for entry in name_position_pairs if (entry[1].find('Не МГТУ') == -1)]
    
    for entry in bmstu_members:
        faculty = detect_faculty(entry[1])
        personal_publications_num.setdefault(entry[0], [0, faculty])
        personal_publications_num[entry[0]][0] += 1
    for single_comb in combinations(bmstu_members, 2):
        sorted_comb = sorted(single_comb, key=lambda tup: tup[0])
        names_comb = (sorted_comb[0][0], sorted_comb[1][0])
        pair_publications_num.setdefault(names_comb, 0)
        pair_publications_num[names_comb] += 1

In [10]:
personal_publications_num = {}
pair_publications_num = {}
stucked_names = core_df['Авторы']
stucked_names.apply(process_group_of_names);

In [11]:
personal_publications_num_df = pd.DataFrame.from_dict(personal_publications_num, orient='index', columns=['pub_num', 'faculty'])
personal_publications_num_df.index.names = ['full_name']
personal_publications_num_df.reset_index(inplace=True)
personal_publications_num_df.loc[:, 'id'] = personal_publications_num_df.apply(lambda row: (row['full_name']), axis=1)


pair_publications_num_df = pd.DataFrame.from_dict(pair_publications_num, orient='index', columns=['pair_pub_num'])
pair_publications_num_df.index.names = ['names_pair']
pair_publications_num_df.reset_index(inplace=True)
get_ids = lambda row: pd.Series([(row['names_pair'][0]),
                                 (row['names_pair'][1])])
pair_publications_num_df[['name1_id', 'name2_id']] = pair_publications_num_df.apply(get_ids, axis=1)

In [12]:
personal_publications_num_df.head()

,full_name,pub_num,faculty,id
0,Филиппов Сергей Сергеевич,1,ФН,Филиппов Сергей Сергеевич
1,Seregina E.V.,6,Other,Seregina E.V.
2,Makarenkov A.M.,5,Other,Makarenkov A.M.
3,Kristya V.I.,15,Other,Kristya V.I.
4,Tun Y.N.,3,Other,Tun Y.N.


In [13]:
pair_publications_num_df.head()

,names_pair,pair_pub_num,name1_id,name2_id
0,"(Makarenkov A.M., Seregina E.V.)",4,Makarenkov A.M.,Seregina E.V.
1,"(Kristya V.I., Tun Y.N.)",3,Kristya V.I.,Tun Y.N.
2,"(Nekhina A.A., Spiridonov I.N.)",1,Nekhina A.A.,Spiridonov I.N.
3,"(Горелик Владимир Семенович, Пятышев А.Ю.)",6,Горелик Владимир Семенович,Пятышев А.Ю.
4,"(Bystrov N.S., Yatsenko P.I.)",1,Bystrov N.S.,Yatsenko P.I.


personal_publications_num_df.to_json('pers_pub.json', orient='records')
pair_publications_num_df[['pair_pub_num', 'name1_id', 'name2_id']].to_json('pair_pub.json', orient='records')

In [14]:
def append_key_data(row):
     return 'Публикаций: ' + str(row.loc['pub_num']) + ', Факультет: ' + row.loc['faculty']

In [15]:
personal_publications_num_df['key_data'] = personal_publications_num_df.apply(
    append_key_data, axis=1)

In [16]:
personal_publications_num_df.head()

,full_name,pub_num,faculty,id,key_data
0,Филиппов Сергей Сергеевич,1,ФН,Филиппов Сергей Сергеевич,"Публикаций: 1, Факультет: ФН"
1,Seregina E.V.,6,Other,Seregina E.V.,"Публикаций: 6, Факультет: Other"
2,Makarenkov A.M.,5,Other,Makarenkov A.M.,"Публикаций: 5, Факультет: Other"
3,Kristya V.I.,15,Other,Kristya V.I.,"Публикаций: 15, Факультет: Other"
4,Tun Y.N.,3,Other,Tun Y.N.,"Публикаций: 3, Факультет: Other"


In [17]:
personal_publications_num_network_format_df = personal_publications_num_df.rename(
    columns={'key_data':'title', 'pub_num':'value',
             'faculty':'group', 'full_name':'label'})[['title', 'value', 'group', 'id', 'label']]
pair_publications_num_network_format_df = pair_publications_num_df.rename(
    columns={'pair_pub_num':'value', 'name1_id':'from', 'name2_id':'to'})[['value', 'from', 'to']]

## Calculation of the circles

In [18]:
faculty_circles_radius = (personal_publications_num_network_format_df.groupby('group')['value'].sum() +
                          10 * personal_publications_num_network_format_df.groupby('group').size()) / (2 * pi)
main_circle_radius = (faculty_circles_radius.size * 10 + faculty_circles_radius.sum() * 2) / (2 * pi)

### calculate centers of the faculty circles

In [19]:
faculties = faculty_circles_radius.index.values.tolist()
faculty_circles_centers = {}
distance_passed = 0
# print(str(main_circle_radius * 2 * pi) + '\n')
for faculty_name in faculties:
    center_angle = (distance_passed + faculty_circles_radius[faculty_name]) / (main_circle_radius * 2 * pi)
    # print(center_angle)
    faculty_circles_centers[faculty_name] = [main_circle_radius * cos(center_angle * 2 * pi),
                                             main_circle_radius * sin(center_angle * 2 * pi)]
    # print(faculty_circles_centers)
    distance_passed += (2 * faculty_circles_radius[faculty_name]) + 10
    # print(distance_passed)

In [20]:
faculty_circles_centers

{'Other': [1211.4426076575996, 906.74506429372],
 'БМТ': [225.77007701550613, 1496.264574062248],
 'ИБМ': [-34.36817527920626, 1512.8114990960523],
 'ИУ': [-307.9926827712704, 1481.5263448986059],
 'Л': [-532.2936368435691, 1416.4897766821734],
 'МТ': [-928.6940922995423, 1194.6995798932871],
 'РК': [-1355.423141215715, 672.7614075778956],
 'РЛ': [-1507.2495326062767, 134.08448727361457],
 'СГН': [-1497.4683707099039, -217.6425555646065],
 'СМ': [-1416.0158186708206, -533.55318811087],
 'ФН': [-168.32207527416233, -1503.8109862095923],
 'Э': [1395.1989315669587, -585.8325226749399],
 'ЮР': [1513.1249730912102, -15.251854870519777]}

In [21]:
faculty_distance_passed = {faculty_name : 0 for faculty_name in faculties}
def calculate_center(row):
    faculty = row['group']
    radius = row['value']
    center_angle = (faculty_distance_passed[faculty] + radius) / (faculty_circles_radius[faculty] * 2 * pi)
    faculty_distance_passed[faculty] += (2 * radius) + 10
    return pd.Series([faculty_circles_radius[faculty] * cos(center_angle * 2 * pi) + faculty_circles_centers[faculty][0], 
            faculty_circles_radius[faculty] * sin(center_angle * 2 * pi) + faculty_circles_centers[faculty][1]])

In [22]:
personal_publications_num_network_format_df[['x', 'y']] = personal_publications_num_network_format_df.apply(
    calculate_center, axis=1) * 100
# personal_publications_num_network_format_df['physics'] = 'false'

In [23]:
personal_publications_num_network_format_df.head()

,title,value,group,id,label,x,y
0,"Публикаций: 1, Факультет: ФН",1,ФН,Филиппов Сергей Сергеевич,Филиппов Сергей Сергеевич,119461.886237,-149081.118331
1,"Публикаций: 6, Факультет: Other",6,Other,Seregina E.V.,Seregina E.V.,218370.164183,91274.502621
2,"Публикаций: 5, Факультет: Other",5,Other,Makarenkov A.M.,Makarenkov A.M.,218334.528615,93374.159419
3,"Публикаций: 15, Факультет: Other",15,Other,Kristya V.I.,Kristya V.I.,218204.981428,96371.241918
4,"Публикаций: 3, Факультет: Other",3,Other,Tun Y.N.,Tun Y.N.,218000.701798,99163.683146


In [24]:
def replace_str(filename, text_to_search, replacement_text):
    with fileinput.FileInput(filename, inplace=True, backup='.bak') as file:
        for line in file:
            print(line.replace(text_to_search, replacement_text), end='')

In [25]:
personal_publications_num_network_format_df.to_json('pers_pub_net.json', orient='records', force_ascii=True)
pair_publications_num_network_format_df.to_json('pair_pub_net.json', orient='records', force_ascii=True)

replace_str('pers_pub_net.json', '"title"', ' title ')
replace_str('pers_pub_net.json', '"value"', ' value ')
replace_str('pers_pub_net.json', '"group"', ' group ')
replace_str('pers_pub_net.json', '"id"', ' id ')
replace_str('pers_pub_net.json', '"label"', ' label ')
# replace_str('pers_pub_net.json', '"physics"', ' physics ')
# replace_str('pers_pub_net.json', '"false"', ' false ')

replace_str('pair_pub_net.json', '"value"', ' value ')
replace_str('pair_pub_net.json', '"from"', ' from ')
replace_str('pair_pub_net.json', '"to"', ' to ')

In [26]:
pair_publications_num_network_format_df.head()

,value,from,to
0,4,Makarenkov A.M.,Seregina E.V.
1,3,Kristya V.I.,Tun Y.N.
2,1,Nekhina A.A.,Spiridonov I.N.
3,6,Горелик Владимир Семенович,Пятышев А.Ю.
4,1,Bystrov N.S.,Yatsenko P.I.


In [27]:
personal_publications_num_network_format_df.head()

,title,value,group,id,label,x,y
0,"Публикаций: 1, Факультет: ФН",1,ФН,Филиппов Сергей Сергеевич,Филиппов Сергей Сергеевич,119461.886237,-149081.118331
1,"Публикаций: 6, Факультет: Other",6,Other,Seregina E.V.,Seregina E.V.,218370.164183,91274.502621
2,"Публикаций: 5, Факультет: Other",5,Other,Makarenkov A.M.,Makarenkov A.M.,218334.528615,93374.159419
3,"Публикаций: 15, Факультет: Other",15,Other,Kristya V.I.,Kristya V.I.,218204.981428,96371.241918
4,"Публикаций: 3, Факультет: Other",3,Other,Tun Y.N.,Tun Y.N.,218000.701798,99163.683146
